## Cracking the Code: Unveiling Resume Sorting Strategies

##### In a world where countless job seekers, myself included, are applying for positions, the sheer volume of resumes overwhelms businesses. Each resumes possesses a distinct structure and content, deviating from any standardized format. Consequently, HR professionals grapple with the daunting task of sifting through these individualistic documents to pinpoint the ideal candidate for a specific job.

##### However, this manual review process is both time-intensive and susceptible to oversights, potentially leading to qualified applicants slipping through the cracks. This tutorial aims to highlight how companies harness the power of machine learning to navigate through multiple resumes. By delving into this process, applicants like myself gain insight into the behind-the-scenes journey of their job appcations. Moreover, understanding this can help us enhance our resumes strategically, optimizing our chances of progressing further in the hiring process.

---


##### To find the best candidate, here's how I assume it works:

1. Recuiters review each resume in the database, seeking those that seamlessly align with the job description.
2. Then selects resumes that go through a ranking process, where relevance to the job description becomes the defining metric.
3. Finally, the key candidate's details such as their name, contact information, and email address are accessed for the next steps of the application process.

##### Addressing this use case involves exploring the realm of Natural Language Processing (NLP). This tutorial will highlight how theses systems parse, match, select, and rank resumes from an extensive pool, all predicted on the basis of the job description.

In [ ]:
! pip install textract
! pip install -U nltk
! pip install pdfminer3
! pip install mammoth
! pip install locationtagger

In [ ]:


# Importing required libraries
import pandas as pd
from google.colab import drive
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import TextConverter
import io
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import en_core_web_sm
nlp = en_core_web_sm.load()
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import mammoth
import locationtagger
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import wordnet
nltk.download('wordnet')
from sklearn.decomposition import TruncatedSVD


In [ ]:
# Making directory of Resumes and Job Description
directory = '/content/drive/MyDrive/'
resume_path = directory + 'Resumes/'
jd_path = directory + 'JD/'


##### Next, let’s extract all the text information from the resumes and job description. Let’s write a function that extracts PDF-format resumes. It can also extract the data from tables. Here we use the PDFResourceManager, PDFPageInterpreter, and TextConverter functions, which are deined in pdfminer3.

In [ ]:
def pdf_extractor(path):
    r_manager = PDFResourceManager()
    output = io.StringIO()
    converter = TextConverter(r_manager, output, laparams=LAParams())
    p_interpreter = PDFPageInterpreter(r_manager, converter)

    with open(path, 'rb') as file:
        for page in PDFPage.get_pages(file, caching=True, check_extractable=True):
            p_interpreter.process_page(page)
            text = output.getvalue()

    converter.close()
    output.close()
    return text


In [ ]:
# A function for reading pdf, docx, doc, and txt files
def read_files(file_path):
    fileTXT = []

    # This loop reads all the files in the specified file_path
    for filename in os.listdir(file_path):
        # For pdf format
        if filename.endswith(".pdf"):
            try:
                fileTXT.append(pdf_extractor(file_path + filename))
            except Exception:
                print('Error reading pdf file: ' + filename)

        # For docx format
        if filename.endswith(".docx"):
            try:
                with open(file_path + filename, "rb") as docx_file:
                    result = mammoth.extract_raw_text(docx_file)
                    text = result.value
                    fileTXT.append(text)
            except IOError:
                print('Error reading .docx file: ' + filename)

        # For doc format
        if filename.endswith(".doc"):
            try:
                text = textract.process(file_path + filename).decode('utf-8')
                fileTXT.append(text)
            except Exception:
                print('Error reading .doc file: ' + filename)

        # For txt format
        if filename.endswith(".txt"):
            try:
                myfile = open(file_path + filename, "rt")
                contents = myfile.read()
                fileTXT.append(contents)
            except Exception:
                print('Error reading .txt file: ' + filename)

    return fileTXT

# resumeTxt is a list containing the resumes of all the candidates.
# Calling the function read_files to read all the resumes
resumeTxt = read_files(resume_path)

# Displaying the first resume
resumeTxt[0]


In [ ]:
# Calling the function read_files to read all the JDs
jdTxt = read_files(jd_path)

# Displaying the content of the first Job Description
jdTxt[0]


In [ ]:
def preprocessing(Txt):
    sw = stopwords.words('english')
    space_pattern = '\s+'
    special_letters = "[^a-zA-Z#]"
    p_txt = []

    for resume in Txt:
        text = re.sub(space_pattern, ' ', resume)  # Removes extra spaces
        text = re.sub(special_letters, ' ', text)  # Removes special characters
        text = re.sub(r'[^\w\s]', '', text)  # Removes punctuations
        text = text.split()  # Splits the words in a text
        text = [word for word in text if word.isalpha()]  # Keeps alphabetic words
        text = [w for w in text if w not in sw]  # Removes stopwords
        text = [item.lower() for item in text]  # Lowercases the words
        p_txt.append(" ".join(text))  # Joins all the words back

    return p_txt


In [ ]:
# Calling the function preprocessing to clean all the resumes
p_resumeTxt = preprocessing(resumeTxt)

# Displaying the first pre-processed resume
p_resumeTxt[0]


In [ ]:
# Calling the function preprocessing to clean all the job descriptions
jds = preprocessing(jdTxt)

# Displaying the first pre-processed job description
jds[0]


In [ ]:
# Combining Resumes and Job Description for finding TF-IDF and Cosine Similarity
TXT = p_resumeTxt + jds

# Finding TF-IDF score of all the resumes and JDs.
tv = TfidfVectorizer(max_df=0.85, min_df=10, ngram_range=(1,3))

# Converting TF-IDF to a DataFrame
tfidf_wm = tv.fit_transform(TXT)
tfidf_tokens = tv.get_feature_names()
df_tfidfvect1 = pd.DataFrame(data=tfidf_wm.toarray(), columns=tfidf_tokens)

print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect1[0:10])


## Feature Reduction

In [ ]:
# Defining transformation
dimrec = TruncatedSVD(n_components=30, n_iter=7, random_state=42)
transformed = dimrec.fit_transform(df_tfidfvect1)

# Converting transformed vector to list
vl = transformed.tolist()

# Converting list to DataFrame
fr = pd.DataFrame(vl)

print('SVD Feature Vector')
print(fr[0:10])


## Model Building

In [ ]:
# Calculating Cosine Similarity between JDs and resumes to find out which resume is the best fit for a job description
similarity = cosine_similarity(df_tfidfvect1[0:len(resumeTxt)], df_tfidfvect1[len(resumeTxt):])

# Column names for job description
abc = [f"JD {i}" for i in range(1, len(jds) + 1)]

# DataFrame of similarity score
Data = pd.DataFrame(similarity, columns=abc)

print('\nCosine Similarity\n')
print(Data[0:10])


In [ ]:
# DataFrame of original resume
t = pd.DataFrame({'Original Resume': resumeTxt})
dt = pd.concat([Data, t], axis=1)  # Assuming a missing closing parenthesis after axis=1


In [ ]:
import re

def number(text):
    pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*)')
    pt = pattern.findall(text)
    pt = [re.sub(r'[,.]', '', ah) for ah in pt if len(re.sub(r'[()\-.,\s+]', '', ah)) > 9]
    pt = [re.sub(r'\D$', '', ah).strip() for ah in pt]
    pt = [ah for ah in pt if len(re.sub(r'\D', '', ah)) <= 15]

    for ah in list(pt):
        if len(ah.split('-')) > 3:
            continue
        for x in ah.split("-"):
            try:
                if x.strip()[-4:].isdigit():
                    if int(x.strip()[-4:]) in range(1900, 2100):
                        pt.remove(ah)
            except: pass

    numbers = list(set(pt))
    return numbers


In [ ]:
import re

def email_ID(text):
    # Compile helps us define a pattern for matching in the text
    pattern = re.compile(r'[A-Za-z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
    return pattern.findall(str(text))


In [ ]:
# Calling the email_ID function to get the list of candidate’s e-mails
dt['E-Mail ID'] = dt['Original Resume'].apply(lambda x: email_ID(x))

print("Extracting e-mail from dataframe columns:")
print(dt['E-Mail ID'][0:5])


In [ ]:
import re

def rm_number(text):
    try:
        pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*)')
        pt = pattern.findall(text)
        pt = [re.sub(r'[,.]', '', ah) for ah in pt if len(re.sub(r'[()\-.,\s+]', '', ah)) > 9]
        pt = [re.sub(r'\D$', '', ah).strip() for ah in pt]
        pt = [ah for ah in pt if len(re.sub(r'\D', '', ah)) <= 15]

        for ah in list(pt):
            if len(ah.split('-')) > 3:
                continue
            for x in ah.split("-"):
                try:
                    if x.strip()[-4:].isdigit():
                        if int(x.strip()[-4:]) in range(1900, 2100):
                            pt.remove(ah)
                except:
                    pass

        number = pt
        number = set(number)
        number = list(number)

        for i in number:
            text = text.replace(i, " ")

        return text
    except:
        pass


In [ ]:
# Calling the function rm_number to remove the phone number
dt['Original'] = dt['Original Resume'].apply(lambda x: rm_number(x))


In [ ]:
import re

def rm_email(text):
    try:
        email = None
        # compile helps define a pattern for matching it in the text
        pattern = re.compile('[\w\.-]+@[\w\.-]+')
        # findall finds the pattern defined in compile
        pt = pattern.findall(text)
        email = pt
        email = set(email)
        email = list(email)

        for i in email:
            # replace will replace a given string with another
            text = text.replace(i, " ")

        return text
    except:
        pass


In [ ]:
# Calling the function rm_email to remove the e-mails
dt['Original'] = dt['Original'].apply(lambda x: rm_email(x))
print("Extracting numbers from dataframe columns:")
print(dt['Original'][0:5])


In [ ]:
# Function to remove emails to extract the year of experience and name of a candidate
def rm_email(text):
    try:
        email = None
        # compile helps us to define a pattern for matching it in the text
        pattern = re.compile('[\w\.-]+@[\w\.-]+')
        # findall finds the pattern defined in compile
        pt = pattern.findall(text)
        email = pt
        email = set(email)
        email = list(email)
        for i in email:
            # replace will replace a given string with another
            text = text.replace(i, " ")
        return text
    except:
        pass


In [ ]:
# Calling the function rm_email to remove the e-mails
dt['Original'] = dt['Original'].apply(lambda x: rm_email(x))
print("Extracting numbers from dataframe columns:")
print(dt['Original'][0:5])


In [ ]:
# Function to get candidates name

import nltk
import re
from nltk.corpus import wordnet

def person_name(text):
    # Tokenizes whole text to sentences
    Sentences = nltk.sent_tokenize(text)
    t = []
    for s in Sentences:
        # Tokenizes sentences to words
        t.append(nltk.word_tokenize(s))
    # Tags a word with its part of speech
    words = [nltk.pos_tag(token) for token in t]
    n = []
    for x in words:
        for l in x:
            # match matches the pos tag of a word to a given tag here
            if re.match('[NN.*]', l[1]):
                n.append(l[0])
    cands = []
    for nouns in n:
        if not wordnet.synsets(nouns):
            cands.append(nouns)
    cand = ' '.join(cands[:1])
    return cand


In [ ]:
# Calling the function name to extract the name of a candidate
dt['Candidate\'s Name'] = dt['Original'].apply(lambda x: person_name(x))
print("Extracting names from dataframe columns:")
print(dt['Candidate\'s Name'][0:5])


In [ ]:
# Function to find the years of experience
def exp(text):
    try:
        e = []
        p = 0
        text = text.lower()

        # Searches a pattern text string similar to the given pattern
        pt1 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,7}experience(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,7}", text)
        if pt1 is not None:
            # groups all the string found in match
            p = pt1.group()

        # Searches a pattern text string similar to the given pattern
        pt2 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,2}year(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,2}", text)
        if pt2 is not None:
            # groups all the string found in match
            p = pt2.group()

        # Searches a pattern text string similar to the given pattern
        pt3 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,2}years(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,2}", text)
        if pt3 is not None:
            # groups all the string found in match
            p = pt3.group()

        if p == 0:
            return 0

        # findall finds the pattern defined in compile
        ep = re.findall('[0-9]{1,2}', p)
        ep_int = list(map(int, ep))

        # this for loop is for filtering and then appending string containing years of experience
        for a in ep:
            for b in ep_int:
                if len(a) <= 2 and b < 30:
                    e.append(a)

        ep = ''.join(e[0])

        # findall finds the pattern defined in compile
        p1 = re.findall('[0-9]{1,2}.[0-9]{1,2}', p)
        exp = []
        if not p1:
            exp.append(ep)
            exp = ''.join(ep)
        else:
            exp.append(p1)
            exp = ''.join(p1)
    except:
        exp = 0

    return exp


In [ ]:
# Calling the function exp to extract the year of experience of the candidate
dt['Experience'] = dt['Original'].apply(lambda x: exp(x))
print("Extracting e-mail from dataframe columns:")
print(dt['Experience'])


In [ ]:
# Importing a file of pre-defined skills
skills = pd.read_excel('/content/drive/MyDrive/skills.xlsx')
skills = skills.values.flatten().tolist()

i = 0
skill = []

for z in skills:
    r = z.lower()
    skill.append(r)
    i += 1


In [ ]:
# Function to extract skills from candidate's resume
def skills(text):
    sw = set(nltk.corpus.stopwords.words('english'))
    tokens = nltk.tokenize.word_tokenize(text)

    # Remove the punctuation
    ft = [w for w in tokens if w.isalpha()]

    # Remove the stop words
    ft = [w for w in tokens if w not in sw]

    # Generate bigrams and trigrams (like Machine Learning)
    n_grams = list(map(' '.join, nltk.everygrams(ft, 2, 3)))

    fs = set()

    # Check for each token in our skills database
    for token in ft:
        if token.lower() in skill:
            fs.add(token)

    # Check for each bigram and trigram in our skills database
    for ngram in n_grams:
        if ngram.lower() in skill:
            fs.add(ngram)

    return fs


In [ ]:
# Calling the function skills to extract the skills of a candidate
dt['Skills'] = dt['Original'].apply(lambda x: skills(x))
print("Extracting Person Name from dataframe columns:")
print(dt['Skills'])


In [ ]:
# Function to extract Location
def location(text):
    place_entity = locationtagger.find_locations(text=text)
    return place_entity.cities


In [ ]:
# Calling the function location to extract the location of a candidate
dt['Location'] = dt['Resume'].apply(lambda x: location(x))
print("Extracting cities from dataframe columns:")
print(dt['Location'])


In [ ]:
# Function to extract Company name
def CompanyName(text):
    # for tagging each entity with its labels
    tokens = nlp(str(text))
    x = []

    # for loop for extracting company names
    for ent in tokens.ents:
        if ent.label_ == 'ORG':
            return ent.text


In [ ]:
# Calling the function CompanyName to extract past companies of a candidate
dt['Company Name'] = dt['Original Resume'].apply(lambda x: CompanyName(x))
print("Extracting Person Name from dataframe columns:")
print(dt['Company Name'])


In [ ]:
# Final result for Project Manager profile
pm = dt[['Project Manager', 'Candidate\'s Name', 'Phone No.', 'E-Mail ID', 'Skills', 'Experience', 'Location', 'Company Name']]
pm = pm.sort_values(by='Project Manager', ascending=False)
print(pm[0:10])


In [ ]:
# Final Result for Business Analyst
ba = dt[['Business Analyst', 'Candidate\'s Name', 'Phone No.', 'E-Mail ID', 'Skills', 'Experience', 'Location', 'Company Name']]
ba = ba.sort_values(by='Business Analyst', ascending=False)
print(ba[0:10])


In [ ]:
# Final Result for Java Developer
jad = dt[['Java Developer', 'Candidate\'s Name', 'Phone No.', 'E-Mail ID', 'Skills', 'Experience', 'Location', 'Company Name']]
jad = jad.sort_values(by='Java Developer', ascending=False)
print(jad[0:10])


## Visualization

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create and generate a word cloud image for the best candidate for Project Manager
wordcloud = WordCloud(width=800, height=500, background_color='white', min_font_size=10).generate(resumeTxt[17])

# Display the generated image
plt.figure(figsize=(20, 5), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create and generate a word cloud image for the best candidate for Business Analyst
wordcloud = WordCloud(width=800, height=500, background_color='white', min_font_size=10).generate(resumeTxt[12])

# Display the generated image
plt.figure(figsize=(20, 5), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create and generate a word cloud image for the best candidate for Java Developer
wordcloud = WordCloud(width=800, height=500, background_color='white', min_font_size=10).generate(resumeTxt[23])

# Display the generated image
plt.figure(figsize=(20, 5), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create and generate a word cloud of the job description for a Java Developer
wordcloud = WordCloud(width=800, height=500, background_color='white', min_font_size=10).generate(jds[0])

# Display the generated image
plt.figure(figsize=(20, 5), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()
